In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

In [2]:
df = pd.read_csv('user_song_rating.csv')

In [3]:
df.head()

,uid,song_id,rating
0,168529700,4068248,2
1,168529682,1719574,2
2,168529704,23491875,1
3,153901397,4310520,2
4,168529900,4117304,2


In [4]:
type(df.rating[0])

numpy.int64

In [5]:
df.describe()

,uid,song_id,rating
count,3.340017e+07,3.340017e+07,3.340017e+07
mean,1.605094e+08,6.155927e+06,1.525855e+00
std,3.336365e+07,7.138677e+06,7.403400e-01
min,1.233300e+04,1.400000e+01,1.000000e+00
25%,1.679196e+08,7.928260e+05,1.000000e+00
50%,1.683411e+08,4.304595e+06,1.000000e+00
75%,1.687034e+08,7.010594e+06,2.000000e+00
max,1.692623e+08,2.147484e+09,5.000000e+00


Check number of unique users and number of unique songs

In [10]:
print "number of unique users is %i, and number of unique songs is %i" %(df.uid.nunique(), df.song_id.nunique())

number of unique users is 590013, and number of unique songs is 1568305


In [11]:
down_freq = pd.read_csv("down_freq.csv",dtype = {"uid" : np.int64, "song_id": np.int64})

In [5]:
down_freq.head()

,uid,song_id,0
0,104057,705790,1
1,104057,1009976,1
2,104057,2926987,1
3,104057,2939508,1
4,104057,2975785,1


In [9]:
s1 = set(df.uid.unique())
s2 = set(down_freq.uid.unique())

In [10]:
print s2.issubset(s2)

True


In [11]:
s1 = set(df.song_id.unique())
s2 = set(down_freq.song_id.unique())

In [12]:
print s2.issubset(s2)

True


In [54]:
df.isnull().any()

uid        False
song_id    False
rating     False
dtype: bool

In [55]:
down_freq.isnull().any()

uid        False
song_id    False
0          False
dtype: bool

In [12]:
play_and_down = pd.merge(df, down_freq, how = "outer")

In [48]:
play_and_down.shape

(34779359, 4)

In [27]:
df.shape

(33400166, 3)

In [13]:
play_and_down.rename(columns = {'0': "down_freq"},inplace= True)

In [14]:
play_and_down.head()

,uid,song_id,rating,down_freq
0,168529700,4068248,2.0,NaN
1,168529682,1719574,2.0,NaN
2,168529704,23491875,1.0,NaN
3,153901397,4310520,2.0,NaN
4,168529900,4117304,2.0,NaN


In [15]:
mask = ~ play_and_down.down_freq.isnull()

In [16]:
mask.head(10)

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
9    False
Name: down_freq, dtype: bool

In [13]:
mask.sum()

4021810

In [14]:
rating_with_down = play_and_down.rating[mask]

In [15]:
rating_with_down.describe()

count    2.642617e+06
mean     2.104649e+00
std      1.018510e+00
min      1.000000e+00
25%      1.000000e+00
50%      2.000000e+00
75%      2.000000e+00
max      5.000000e+00
Name: rating, dtype: float64

In [16]:
play_and_down.at[mask, 'rating'] = 5

In [17]:
play_and_down.describe()

,uid,song_id,rating,down_freq
count,3.477936e+07,3.477936e+07,3.477936e+07,4.021810e+06
mean,1.604142e+08,6.126362e+06,1.883620e+00,1.310353e+00
std,3.354592e+07,7.109884e+06,1.299927e+00,2.961040e+00
min,1.233300e+04,1.400000e+01,1.000000e+00,1.000000e+00
25%,1.679174e+08,7.846490e+05,1.000000e+00,1.000000e+00
50%,1.683387e+08,4.292850e+06,1.000000e+00,1.000000e+00
75%,1.687031e+08,6.997970e+06,2.000000e+00,1.000000e+00
max,1.692623e+08,2.147484e+09,5.000000e+00,1.275000e+03


In [18]:
play_and_down.drop("down_freq", axis = 1, inplace= True)

In [19]:
play_and_down.head()

,uid,song_id,rating
0,168529700,4068248,2.0
1,168529682,1719574,2.0
2,168529704,23491875,1.0
3,153901397,4310520,2.0
4,168529900,4117304,2.0


In [20]:
play_and_down.to_csv("user_song_rating_with_down.csv", index = False)

In [11]:
import graphlab
sf = graphlab.SFrame(df)

In [ ]:
item_item_rec = graphlab.recommender.item_similarity_recommender.create(sf,
                                                                        user_id='uid',
                                                                        item_id='song_id',
                                                                        target = 'rating')

Recsys training: model = item_similarity

Preparing data set.

Data has 33400166 observations with 590013 users and 1568305 items.

Data prepared in: 37.5296s

Training model from provided data.

...

| 1h 31m                              | 5.25             | 85071           |

| 1h 31m                              | 5.25             | 85074           |

| 1h 31m                              | 5.25             | 85074           |

| 1h 31m                              | 5.25             | 85078           |

| 1h 31m                              | 5.25             | 85078           |